In [6]:
from datasets import load_dataset
import pandas as pd

# Load StereoSet dataset
stereoset = load_dataset("stereoset")

# Extract and examine intrasentence examples (these are better for training)
intrasentence = stereoset["validation"]["intrasentence"]

# Filter for racial bias examples
racial_examples = [
    example for example in intrasentence 
    if example["bias_type"] == "race"
]

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(racial_examples)

print(f"Found {len(racial_examples)} examples of racial bias")
print(df.head())

NameError: name 'intersentence' is not defined

In [ ]:
# Drop rows with missing values
df.dropna(inplace=True)

# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(df.head())

In [ ]:
# Function to create balanced instruction-response pairs
def create_training_pair(example):
    # Extract the context (sentence with [BLANK])
    context = example["context"]
    
    # Get the stereotype and anti-stereotype completions
    stereotype_option = next(opt for opt in example["sentences"] 
                            if opt["gold_label"] == "stereotype")
    antistereotype_option = next(opt for opt in example["sentences"] 
                                if opt["gold_label"] == "anti-stereotype")
    
    # Create a balanced prompt-response pair
    prompt = f"Complete this sentence in a non-stereotypical way: {context}"
    
    # Use the anti-stereotype option as the target response
    response = context.replace("[BLANK]", antistereotype_option["sentence"])
    
    return {
        "text": f"### Instruction: {prompt}\n\n### Response: {response}"
    }

# Create training examples
training_examples = [create_training_pair(example) for example in racial_examples]

# Convert to Hugging Face dataset format
from datasets import Dataset
training_dataset = Dataset.from_pandas(pd.DataFrame(training_examples))

: 